In [1]:
# This file contains definitions for graphs/networks and sub-elements, such as nodes and edges

# Definition of an edge
class edge:
    # Initialization function requires a start and end node. All other parameters take default values to begin with
    def __init__(self, startNode, endNode):
        self.startNode = startNode
        self.endNode = endNode
        self.capacity = 0
        self.cost = 0
        self.utilization = 0
        self.category = 0

    # Function to set or change capacity
    def setCapacity(self, capacity):
        self.capacity = capacity

    # Function to set or change cost
    def setCost(self, cost):
        self.cost = cost

    # Function to set or change utilization
    def setUtilization(self, utilization):
        self.utilization = utilization

    # Function to set or change category
    def setCategory(self, category):
        self.category = category

    # Function to print edge
    def printEdge(self):
        print(self.startNode, "->", self.endNode, ", cap ", self.capacity,
              ", cost ", self.cost, ", util ", self.utilization)

    # Function to print edge in a simpler (succinct) manner
    def printEdgeSimple(self):
        print(self.startNode, "->", self.endNode)


class node:
    # Initialization function requires a nodeID. All other parameters take default values to begin with
    def __init__(self, nodeID):
        self.nodeID = nodeID
        self.category = 0
        self.distance = None
        self.parent = None
        self.edgesIn = {}
        self.edgesOut = {}

    # Function to set or change category
    def setCategory(self, category):
        self.category = category

    # Function to set or change distance
    def setDistance(self, distance):
        self.distance = distance

    # Function to set or change parent
    def setParent(self, parent):
        self.parent = parent

    # Function to add/overwrite inward edge
    def addEdgeIn(self, edgeIn):
        self.edgesIn[edgeIn.startNode] = edgeIn

    # Function to delete inward edge
    def delEdgeIn(self, edgeIn):
        if edgeIn.startNode in self.edgesIn.keys():
            self.edgesIn.pop(edgeIn.startNode)

    # Function to add/overwrite outward edge
    def addEdgeOut(self, edgeOut):
        self.edgesOut[edgeOut.endNode] = edgeOut

    # Function to delete outward edge
    def delEdgeOut(self, edgeOut):
        if edgeOut.endNode in self.edgesOut.keys():
            self.edgesOut.pop(edgeOut.endNode)

    # Function to print node
    def printNode(self):
        print(self.nodeID)


class graph:
    # Initialization function creates an empty dictionary of nodes, with empty source and target variables
    def __init__(self):
        self.nodes = {}
        self.sourceID = None
        self.targetID = None

    # Function to add/overwrite a node
    def addNode(self, addNode):
        self.nodes[addNode] = node(addNode)

    # Function to add a node only if node already doesn't exist
    def safeAddNode(self, addNode):
        if not addNode in self.nodes.keys():
            self.nodes[addNode] = node(addNode)

    # Function to add/overwrite an edge
    def addEdge(self, startNode, endNode, capacity=0, cost=0, utilization=0, category=0, bidirectional=False):
        # Create nodes if missing
        if not startNode in self.nodes.keys():
            self.nodes[startNode] = node(startNode)
        if not endNode in self.nodes.keys():
            self.nodes[endNode] = node(endNode)
        # Create edges and add edges to nodes
        edgeToAdd = edge(startNode, endNode)
        edgeToAdd.setCapacity(capacity)
        edgeToAdd.setCost(cost)
        edgeToAdd.setUtilization(utilization)
        edgeToAdd.setCategory(category)
        self.nodes[startNode].addEdgeOut(edgeToAdd)
        self.nodes[endNode].addEdgeIn(edgeToAdd)
        # For bidirectional edge, create reverse edge and add to nodes
        if bidirectional:
            edgeToAdd = edge(endNode, startNode)
            edgeToAdd.setCapacity(capacity)
            edgeToAdd.setCost(cost)
            edgeToAdd.setUtilization(utilization)
            edgeToAdd.setCategory(category)
            self.nodes[startNode].addEdgeOut(edgeToAdd)
            self.nodes[endNode].addEdgeOut(edgeToAdd)
            self.nodes[startNode].addEdgeIn(edgeToAdd)

    # Function to delete an edge
    def delEdge(self, startNode, endNode, bidirectional=False):
        edgeToDelete = edge(startNode, endNode)
        if startNode in self.nodes.keys():
            self.nodes[startNode].delEdgeOut(edgeToDelete)
        if endNode in self.nodes.keys():
            self.nodes[endNode].delEdgeIn(edgeToDelete)
        if bidirectional:
            edgeToDelete = edge(endNode, startNode)
            if startNode in self.nodes.keys():
                self.nodes[startNode].delEdgeIn(edgeToDelete)
            if endNode in self.nodes.keys():
                self.nodes[endNode].delEdgeOut(edgeToDelete)

    # Function to delete a node
    def delNode(self, remNode):
        # Delete all of the nodes edges from adjacent nodes
        if remNode in self.nodes.keys():
            for ed in self.nodes[remNode].edgesIn:
                if ed in self.nodes.keys():
                    self.nodes[ed].delEdgeOut(edge(ed, remNode))
            for ed in self.nodes[remNode].edgesOut:
                if ed in self.nodes.keys():
                    self.nodes[ed].delEdgeIn(edge(remNode, ed))

        # Delete node itself (and all edges in and out that it holds)
        self.nodes.pop(remNode)

    # Function to print each node in the graph, with all of its incoming & outgoing edges
    def printGraph(self):
        for no in self.nodes:
            nod = self.nodes[no]
            print("Node",end=" ")
            nod.printNode()
            print("Edges In")
            for ed in nod.edgesIn.values():
                ed.printEdge()
            print("Edges Out")
            for ed in nod.edgesOut.values():
                ed.printEdge()

    # Function to print the graph in a simpler (almost edge list) manner
    def printGraphSimple(self):
        for no in self.nodes:
            nod = self.nodes[no]
            nod.printNode()
            for ed in nod.edgesOut.values():
                ed.printEdgeSimple()

    # Function to clear parents across all nodes
    def clearParents(self):
        for no in self.nodes:
            self.nodes[no].parent = None

    # Function to clear category across all nodes
    def clearCategory(self):
        for no in self.nodes:
            self.nodes[no].category = 0

    # Function to print from a given node, upwards through parents
    def printThroughParent(self, no):
        nod = self.nodes[no]
        nod.printNode()
        if not nod.parent == None:
            self.printThroughParent(nod.parent)


In [2]:
def inputLine(g, l, delimit=" ", inputType="directedArc"):
    if inputType == "directedArc":
        params = l.split(delimit)
        if len(params) == 4:
            g.safeAddNode(params[0])
            g.safeAddNode(params[1])
            g.addEdge(params[0], params[1], cost=int(
                params[2]), capacity=int(params[3]))
    elif inputType == "bidirectionalArc":
        params = l.split(delimit)
        if len(params) == 4:
            g.safeAddNode(params[0])
            g.safeAddNode(params[1])
            g.addEdge(params[0], params[1], cost=int(
                params[2]), capacity=int(params[3]))
            g.addEdge(params[1], params[0], cost=int(
                params[2]), capacity=int(params[3]))
    elif inputType == "directedEdge":
        params = l.split(delimit)
        if len(params) == 2:
            g.safeAddNode(params[0])
            g.safeAddNode(params[1])
            g.addEdge(params[0], params[1])
    elif inputType == "bidirectionalEdge":
        params = l.split(delimit)
        if len(params) == 2:
            g.safeAddNode(params[0])
            g.safeAddNode(params[1])
            g.addEdge(params[0], params[1])
            g.addEdge(params[1], params[0])
    return g

# Function to process one line of input and return an edge


def inputLineEdge(edgeList, l, delimit=" ", inputType="directedArc"):
    if inputType == "directedArc":
        params = l.split(delimit)
        if len(params) == 4:
            ed = edge(params[0], params[1])
            ed.setCost(int(params[2]))
            ed.setCapacity(int(params[3]))
            edgeList.append(ed)
    elif inputType == "bidirectionalArc":
        params = l.split(delimit)
        if len(params) == 4:
            ed = edge(params[0], params[1])
            ed.setCost(int(params[2]))
            ed.setCapacity(int(params[3]))
            edgeList.append(ed)
            ed2 = edge(params[1], params[0])
            ed2.setCost(int(params[2]))
            ed2.setCapacity(int(params[3]))
            edgeList.append(ed2)
    elif inputType == "directedEdge":
        params = l.split(delimit)
        if len(params) == 2:
            ed = edge(params[0], params[1])
            edgeList.append(ed)
    elif inputType == "bidirectionalEdge":
        params = l.split(delimit)
        if len(params) == 2:
            ed = edge(params[0], params[1])
            edgeList.append(ed)
            ed2 = edge(params[1], params[0])
            edgeList.append(ed2)
    return edgeList

# Function to read file and create a graph output


def inputFile(file='.\Session 7.txt', delimit=" ", inputType="directedArc"):
    file1 = open(file, 'r')
    Lines = file1.readlines()
    count = 0
    g = graph()
    for lin in Lines:
        g = inputLine(g, lin.strip(), delimit, inputType)
        count += 1
    return g

# Function to read file and create an edge list


def inputFileEdges(file='.\Session 7.txt', delimit=" ", inputType="directedArc"):
    file1 = open(file, 'r')
    Lines = file1.readlines()
    count = 0
    edgeList = []
    for lin in Lines:
        edgeList = inputLineEdge(edgeList, lin.strip(), delimit, inputType)
        count += 1
    return edgeList

# Function to process one line of input and return the sport name, venue name, start time and end time




In [3]:
# Function to find the closest currently reachable node in the graph


def closest(g):
    min = None
    closeNode = None
    for n in g.nodes.keys():
        nod = g.nodes[n]
        # Check if node has been visited yet
        if nod.category == 1:
            if min == None:
                min = nod.distance
                closeNode = n
            elif nod.distance < min:
                min = nod.distance
                closeNode = n
    return closeNode

# Function to return index of parent of a given node (index) in min heap


def parentIndex(pos):
    if pos == 1:
        return 1
    elif pos % 2 == 0:
        return int(pos/2)
    else:
        return int((pos-1)/2)

# Function to swap 2 elements in a list


def swapPosition(lis, pos1, pos2):
    lis[pos1-1], lis[pos2-1] = lis[pos2-1], lis[pos1-1]
    return lis

# Function to compare 2 elements and return the lower value index


def minPosition(g, lis, pos1, pos2):
    if g.nodes[lis[pos1-1]].distance < g.nodes[lis[pos2-1]].distance:
        return pos1
    else:
        return pos2

# Function to check if heap rules are followed at pos, and make required changes at pos if need be
# For each change made at pos, it will move up and keep checking


def heapRulesUp(g, lis, pos):
    parPos = parentIndex(pos)
    if not minPosition(g, lis, pos, parPos) == parPos:
        lis = swapPosition(lis, pos, parPos)
        lis = heapRulesUp(g, lis, parPos)
    return lis

# Function to move down he heap tree and fill in a gap at pos


def heapRulesDown(g, lis, pos):
    childPos1 = pos*2
    childPos2 = (pos*2)+1
    if len(lis) >= childPos2:
        lowestPos = minPosition(g, lis, childPos1, childPos2)
        lis = swapPosition(lis, pos, lowestPos)
        return heapRulesDown(g, lis, lowestPos)
    elif len(lis) == childPos1:
        lowestPos = childPos1
        lis = swapPosition(lis, pos, lowestPos)
        return heapRulesDown(g, lis, lowestPos)
    else:
        lis = swapPosition(lis, pos, len(lis))
        lis.pop(len(lis)-1)
        if pos < len(lis):
            return heapRulesUp(g, lis, pos)
        else:
            return lis

# Function to push an element into a heap


def heapPush(g, lis, el):
    lis.append(el)
    pos = len(lis)
    return heapRulesUp(g, lis, pos)

# Function to pop an element from a heap


def heapPop(g, lis):
    el = lis[0]
    lis = heapRulesDown(g, lis, 1)
    return el

# Function to convert a list into a heap


def makeHeap(g, lis):
    newLis = []
    for n in lis:
        newLis = heapPush(g, newLis, n)
    return newLis


# Function to print the distances of all nodes in the graph
def distances(g):
    dist = {}
    for n in g.nodes.keys():
        nod = g.nodes[n]
        dist[n] = nod.distance
    print(dist)
    return dist

# Function to implement djikstra's algorithm in O(v^2)


def dijkstra(g, s):
    current = g.nodes[s]
    current.setDistance(0)
    current.setCategory(2)
    count = 1
    while not current == None:
        print("iteration ", count)
        print("parent ", current.nodeID)
        for n in current.edgesOut.keys():
            nod = g.nodes[n]
            newDistance = current.distance+current.edgesOut[n].cost
            if not nod.category == 2:
                if (nod.distance == None) or (nod.distance > newDistance):
                    print("node changed ")
                    nod.printNode()
                    print("distance change ", nod.distance, " to ", newDistance)
                    nod.setDistance(newDistance)
                    nod.setCategory(1)
                    nod.setParent(current.nodeID)
        if not closest(g) == None:
            current = g.nodes[closest(g)]
            current.setCategory(2)
        else:
            current = None
        count += 1
    return distances(g)

# Function to implement djikstra's algorithm in O(elog(v))


def dijkstraMinHeap(g, s):
    heapList = []
    current = g.nodes[s]
    current.setDistance(0)
    current.setCategory(2)
    count = 1
    while not current == None:
        print("iteration ", count)
        print("parent ", current.nodeID)
        for n in current.edgesOut.keys():
            nod = g.nodes[n]
            newDistance = current.distance+current.edgesOut[n].cost
            if not nod.category == 2:
                if (nod.distance == None) or (nod.distance > newDistance):
                    print("node changed ")
                    nod.printNode()
                    print("distance change ", nod.distance, " to ", newDistance)
                    nod.setDistance(newDistance)
                    if nod.category == 0:
                        nod.setCategory(1)
                        heapPush(g, heapList, n)
                    nod.setParent(current.nodeID)
        if len(heapList) > 0:
            current = g.nodes[heapPop(g, heapList)]
            current.setCategory(2)
        else:
            current = None
        count += 1
    return distances(g)


In [4]:

def pathToArray(g, t):
    pathFromT = []
    pathFromT.append(t)
    parent = t.parent
    while not parent == None:
        pathFromT.append(g.nodes[parent])
        parent = g.nodes[parent].parent
    pathToT = []
    i = len(pathFromT)-1
    while i >= 0:
        pathToT.append(pathFromT[i])
        i = i-1
    return pathToT

# Function to implement djikstra's algorithm to get a path (node list) from s to t


def dijkstraOnePath(g, s, t):
    heapList = []
    current = g.nodes[s]
    current.setDistance(0)
    current.setCategory(2)

    while not current == None:
        for n in current.edgesOut.keys():
            nod = g.nodes[n]
            newDistance = current.distance+current.edgesOut[n].cost
            if not nod.category == 2:
                if (nod.distance == None) or (nod.distance > newDistance):
                    nod.setDistance(newDistance)
                    if nod.category == 0:
                        nod.setCategory(1)
                        heapPush(g, heapList, n)
                    nod.setParent(current.nodeID)
        if len(heapList) > 0:
            current = g.nodes[heapPop(g, heapList)]
            current.setCategory(2)
            if current.nodeID == t:
                return pathToArray(g, current)
        else:
            current = None
    return None

# Function to copy an edge


def copyEdge(ed):
    edgeCopy = edge(ed.startNode, ed.endNode)
    edgeCopy.setCapacity(ed.capacity)
    edgeCopy.setCost(ed.cost)
    edgeCopy.setCategory(ed.category)
    edgeCopy.setUtilization(ed.utilization)
    return edgeCopy

# Function to copy a node


def copyNode(n):
    nodeCopy = node(n.nodeID)
    nodeCopy.setCategory(n.category)
    nodeCopy.setDistance(n.distance)
    nodeCopy.setParent(n.parent)
    for ed in sorted(n.edgesIn.keys()):
        edgeToAdd = copyEdge(n.edgesIn[ed])
        nodeCopy.addEdgeIn(edgeToAdd)
    for ed in sorted(n.edgesOut.keys()):
        edgeToAdd = copyEdge(n.edgesOut[ed])
        nodeCopy.addEdgeOut(edgeToAdd)
    return nodeCopy

# Function to copy graph


def copyGraph(g):
    graphCopy = graph()
    for n in sorted(g.nodes.keys()):
        nodeToAdd = copyNode(g.nodes[n])
        graphCopy.nodes[nodeToAdd.nodeID] = nodeToAdd
    return graphCopy

# Function to check if the root path matches the start of comparison path


def rootMatches(root, otherPath):
    rootLength = len(root)
    for i in range(rootLength):
        if not root[i].nodeID == otherPath[i].nodeID:
            return False
    return True

# Function to return subpath after a certain point


def restOfPath(path, start):
    if len(path) > start:
        return path[start:]
    else:
        return []

# Function to remove a path from the given graph and return modified graph


def removePath(g, path):
    if len(path) < 2:
        return g
    else:
        pos = 0
        while pos < (len(path)-1):
            g.delEdge(path[pos].nodeID, path[pos+1].nodeID)
            pos += 1
        return g

# Function to check if the given path isn't present in an array


def notPresent(path, listOfPaths):
    if path == None:
        return False
    for p in listOfPaths:
        if rootMatches(path, p):
            return False
    return True

# Function to calculate the length of a given path


def lengthOfPath(path):
    if len(path) < 2:
        return 0
    else:
        pos = 0
        length = 0
        while pos < (len(path)-1):
            length += path[pos].edgesOut[path[pos+1].nodeID].cost
            pos += 1
        return length

# Function to sort the paths in an array basis path length


def sortPaths(listOfPaths):
    pathLengths = []
    for path in listOfPaths:
        pathLengths.append((lengthOfPath(path), path))
    pathLengths.sort(key=lambda x: x[0])
    sortedPaths = []
    for path in pathLengths:
        sortedPaths.append(path[1])
    return sortedPaths

# Function to implement yen's algo and return the kth shortest path between s and t


def yenKSP(g, s, t, k):
    # Initialise array to hold all shortest paths
    shortestPaths = []
    graphToUse = copyGraph(g)
    shortestPaths.append(dijkstraOnePath(graphToUse, s, t))
    potentialPaths = []
    # Iterate k times, till kth path found
    pathsFound = 1
    while pathsFound < k:
        # From the immediately previous path select each node as spur node one by one
        previousPath = shortestPaths[pathsFound-1]
        previousPathLength = len(previousPath)
        for i in range(previousPathLength-1):
            spurNode = previousPath[i]
            # Root path will be the start of the path being discovered
            rootPath = previousPath[:(i+1)]
            # Create a copy of the original graph
            graphToUse = copyGraph(g)
            # Remove next edge of all previous paths which start with root path
            for path in shortestPaths:
                # Check if root path matches
                if rootMatches(rootPath, path):
                    # Remove the edge that connects spur node to next node in the path
                    pos = len(rootPath)-1
                    graphToUse.delEdge(path[pos].nodeID, path[pos+1].nodeID)
            # Remove all nodes in root path except spur node
            for n in rootPath:
                # Remove node from graph copy
                if not n == spurNode:
                    graphToUse.delNode(n.nodeID)
            # Find shortest path from spur node to target
            nextPath = dijkstraOnePath(graphToUse, spurNode.nodeID, t)
            if not nextPath == None:
                nextPath = rootPath + nextPath[1:]
            # Add the full path (root + rest) to potential paths, if not already present
            if notPresent(nextPath, potentialPaths):
                potentialPaths.append(nextPath)
            # Sort potential paths & pop the shortest path into shortest paths
        if len(potentialPaths) > 0:
            potentialPaths = sortPaths(potentialPaths)
            shortestPaths.append(potentialPaths[0])
            potentialPaths.pop(0)
        else:
            return shortestPaths
        pathsFound += 1

    return shortestPaths

# Function to print a given path


def printPath(path):
    pathLength = lengthOfPath(path)
    print("path length ", pathLength)
    for n in path:
        print(n.nodeID, "->", end="")
    print("")


In [8]:
g1 = inputFile(r".\Session 7.txt")

"""
#Array containing path (of node objects) till node t
arrayPath = pathToArray(g1,"t")
#Dijkstra's shortest path s-t returned as an array
arrayPath = dijkstraOnePath(g1,"0","1")
#Functions to copy graph elements
edgeCopy = copyEdge(ed)
nodeCopy = copyNode(n)
graphCopy = copyGraph(g1)
#Returns all of the k shortest paths in increasing length order
"""
K = 11 #number of paths
shortestPaths = yenKSP(g1,"0","7",K)
#Prints a path given a path array
for i in range(K):
    print('path', i+1)
    printPath(shortestPaths[i])


path 1
path length  10
0 ->4 ->3 ->1 ->6 ->2 ->7 ->
path 2
path length  12
0 ->1 ->6 ->2 ->7 ->
path 3
path length  12
0 ->4 ->3 ->1 ->2 ->7 ->
path 4
path length  12
0 ->4 ->3 ->1 ->6 ->7 ->
path 5
path length  12
0 ->3 ->1 ->6 ->2 ->7 ->
path 6
path length  13
0 ->4 ->5 ->7 ->
path 7
path length  13
0 ->4 ->2 ->7 ->
path 8
path length  14
0 ->1 ->2 ->7 ->
path 9
path length  14
0 ->1 ->6 ->7 ->
path 10
path length  14
0 ->3 ->1 ->2 ->7 ->
path 11
path length  14
0 ->3 ->1 ->6 ->7 ->
